In [48]:
%matplotlib inline

In [49]:
import sys
from os.path import join
import numpy as np

from scipy import stats
from scipy.interpolate import interp2d
from pyproj import Proj, transform

from mtpy.modeling.modem import Model, Data
from mtpy.utils import gis_tools

from mtpy.utils import lib
from mtpy.utils import nc
from mtpy.utils.modem_to_netCDF import converter, lon_lat_grid_spacing, uniform_interior_grid
from mtpy.utils.modem_to_netCDF import median_spacing, interpolated_layer

In [50]:
# Define Data and Model Paths
MT_PATH = '../../../mtpy/examples/model_files/ModEM/'

data = Data()
data.read_data_file(data_fn=join(MT_PATH, 'ModEM_Data.dat'))

# create a model object using the data object and read in model data
model = Model(data_obj=data)
model.read_model_file(model_fn=join(MT_PATH, 'ModEM_Model_File.rho'))


2018-12-18T11:20:10 - Data - INFO - Set rotation angle to 0.0 deg clockwise from N
      -15500.000      -21500.000         -67.000

    0.000



In [51]:
center = data.center_point

# dictionary for resistivity model data
# populate variables

resistivity_data = {
        'x': center.east.item() + (model.grid_east[1:] + model.grid_east[:-1])/
2,
        'y': center.north.item() + (model.grid_north[1:] + model.grid_north[:-1
])/2,
        'z': (model.grid_z[1:] + model.grid_z[:-1])/2,
        'resistivity': np.transpose(model.res_model, axes=(2, 0, 1))
    }



In [52]:
# the get_utm_zone function gives us the utm zone of our source data.

zone_number, is_northern, utm_zone = gis_tools.get_utm_zone(center.lat.item(), center.lon.item())

# wgs84_proj is the lat lon global projection.

wgs84_proj = Proj(init='epsg:4326')
source_proj = Proj('+proj=utm +zone=%d +%s +datum=%s' % (zone_number, 'north' if is_northern else 'south', 'WGS84'))
    
# converter function gives us the transformation between two coordinate

to_wgs84 = converter(source_proj, wgs84_proj)
from_wgs84 = converter(wgs84_proj, source_proj)


In [53]:
# width and height are the median spacing between data we 
# want to interploated in the lat, lon space

center_lon, center_lat, width, height = lon_lat_grid_spacing(center, 
                                                             median_spacing(model.grid_east),
                                                             median_spacing(model.grid_north), 
                                                             to_wgs84)


In [54]:
# to_wgs84 applied on the source data to transform it in lat, lon
lon_list = [to_wgs84(x, y)[0]
                for x in resistivity_data['x']
                for y in resistivity_data['y']]

lat_list = [to_wgs84(x, y)[1]
                for x in resistivity_data['x']
                for y in resistivity_data['y']]

# interpolation_func are the function for layer for each depth 

interpolation_funcs = [interpolated_layer(resistivity_data['x'],
                                              resistivity_data['y'],
                                              resistivity_data['resistivity'][z_index, :, :])
                           for z_index in range(resistivity_data['z'].shape[0])
                      ]

# uniform_interior_grid make a regular grid in lat/lon space.

result = {
        'longitude': uniform_interior_grid(sorted(lon_list), width, center_lon),
        'latitude': uniform_interior_grid(sorted(lat_list), height, center_lat),
        'depth': resistivity_data['z']
         }

result['resistivity'] = np.zeros(tuple(result[key].shape[0]
                                           for key in ['depth', 'latitude', 'longitude']))


In [55]:
def uniform_layer(interp_func, latitudes, longitudes):
    """
    Calculate the interpolated values for the layer.
    """
    lats, lons = latitudes.shape[0], longitudes.shape[0]

    result = np.zeros((lats, lons))
    for j in range(lons):
        for i in range(lats):
            lon, lat = longitudes[j], latitudes[i]
            x, y = from_wgs84(lon, lat)

            result[i, j] = interp_func(x, y)

    return result



In [35]:
for z_index in range(result['depth'].shape[0]):
    print 'layer #', z_index + 1
    result['resistivity'][z_index, :, :] = uniform_layer(interpolation_funcs[z_index],
                                                             result['latitude'], result['longitude'])

    nc.write_resistivity_grid('wgs84.nc', 4326,
                              result['latitude'], result['longitude'], result['depth'],
                              result['resistivity'], z_label='depth')



layer # 1
layer # 2
layer # 3
layer # 4
layer # 5
layer # 6
layer # 7
layer # 8
layer # 9
layer # 10
layer # 11
layer # 12
layer # 13
layer # 14
layer # 15
layer # 16
layer # 17
layer # 18
layer # 19
layer # 20
layer # 21
layer # 22
layer # 23
layer # 24
layer # 25
layer # 26
layer # 27
layer # 28
layer # 29
layer # 30
layer # 31
layer # 32
layer # 33
layer # 34
layer # 35
layer # 36
layer # 37
layer # 38
layer # 39
layer # 40
layer # 41
layer # 42
layer # 43
layer # 44
layer # 45
layer # 46
layer # 47
layer # 48
layer # 49
layer # 50
layer # 51
layer # 52
layer # 53
layer # 54
layer # 55
layer # 56
layer # 57
layer # 58
layer # 59
layer # 60
layer # 61
layer # 62
layer # 63
layer # 64
layer # 65
layer # 66
layer # 67
layer # 68
layer # 69
layer # 70
layer # 71
layer # 72
layer # 73
layer # 74
layer # 75
layer # 76
layer # 77
layer # 78
layer # 79
layer # 80
layer # 81
layer # 82
layer # 83
layer # 84
layer # 85
layer # 86
layer # 87
layer # 88
layer # 89
layer # 90
layer # 91
layer # 